In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt

## Reading in the data files

#### Vaclog 1

In [2]:
#reading vaclogger measurement file
vaclog1 = pd.read_csv("vaclog s7 desorption", sep="\t")
vaclog1.head()

#adding an elapsed time column
vac_timestamps1 = pd.to_datetime(vaclog1["Time"],format="%d/%m/%Y %H:%M:%S")
runtime1 = (vac_timestamps1-vac_timestamps1[0]).dt.total_seconds()    
vaclog1.insert(2,"Elapsed time",runtime1)
# vaclog1.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog1["Time"] = vac_timestamps1.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog1["Time"] = pd.to_datetime(vaclog1["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog1.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2023-02-24 10:52:44,0.0,82.448,1.170000e-09,1.410000e-09,4.996000e-09,2.520000e-07,1.1,466.0,-5.050,4274.281,NaN,NaN
1,NaN,2023-02-24 10:52:51,7.0,82.448,1.170000e-09,1.410000e-09,4.996000e-09,2.529000e-07,1.1,466.0,-5.275,4274.281,NaN,NaN
2,NaN,2023-02-24 10:52:58,14.0,82.447,1.170000e-09,1.400000e-09,4.996000e-09,2.526000e-07,1.1,466.0,-5.275,4274.506,NaN,NaN
3,NaN,2023-02-24 10:53:04,20.0,82.447,1.170000e-09,1.410000e-09,4.996000e-09,2.526000e-07,1.1,466.0,-5.050,4274.618,NaN,NaN
4,NaN,2023-02-24 10:53:11,27.0,82.452,1.170000e-09,1.430000e-09,4.996000e-09,2.529000e-07,1.1,466.0,-5.050,4274.730,NaN,NaN


#### Vaclog 2

In [3]:
#reading vaclogger measurement file
vaclog2 = pd.read_csv("vaclog s6 ads isotherm", sep="\t")
vaclog2.head()

#adding an elapsed time column
vac_timestamps2 = pd.to_datetime(vaclog2["Time"],format="%d/%m/%Y %H:%M:%S")
runtime2 = (vac_timestamps2-vac_timestamps2[0]).dt.total_seconds()    
vaclog2.insert(2,"Elapsed time",runtime2)
# vaclog2.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog2["Time"] = vac_timestamps2.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog2["Time"] = pd.to_datetime(vaclog2["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog2

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2023-03-10 11:41:44,0.0,96.842,1.030000e-09,1.290000e-09,4.996000e-09,2.670000e-07,1.1,292.0,-5.050,4288.311,NaN,NaN
1,NaN,2023-03-10 11:41:51,7.0,96.846,1.030000e-09,1.290000e-09,4.996000e-09,2.673000e-07,1.1,292.0,-5.163,4288.984,NaN,NaN
2,NaN,2023-03-10 11:41:57,13.0,96.846,1.030000e-09,1.290000e-09,4.996000e-09,2.673000e-07,1.1,292.0,-4.938,4289.657,NaN,NaN
3,NaN,2023-03-10 11:42:04,20.0,96.847,1.030000e-09,1.280000e-09,4.996000e-09,2.676000e-07,1.1,292.0,-5.050,4290.555,NaN,NaN
4,NaN,2023-03-10 11:42:11,27.0,96.846,1.030000e-09,1.280000e-09,4.996000e-09,2.676000e-07,1.1,292.0,-5.163,4291.341,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,NaN,2023-03-10 18:24:38,24174.0,90.608,0.000000e+00,0.000000e+00,1.670000e-06,3.036000e-07,1.1,242.0,-5.163,4304.921,NaN,NaN
3483,NaN,2023-03-10 18:24:45,24181.0,90.609,0.000000e+00,0.000000e+00,1.668000e-06,3.039000e-07,1.1,242.0,-5.050,4305.145,NaN,NaN
3484,NaN,2023-03-10 18:24:51,24187.0,90.609,0.000000e+00,0.000000e+00,1.670000e-06,3.036000e-07,1.1,242.0,-5.050,4305.145,NaN,NaN
3485,NaN,2023-03-10 18:24:58,24194.0,90.609,0.000000e+00,0.000000e+00,1.672000e-06,3.036000e-07,1.1,241.0,-5.163,4305.370,NaN,NaN


#### Vaclog 3

In [4]:
#reading vaclogger measurement file
vaclog3=pd.read_csv("vaclog s2 desorption stat vac", sep="\t")
vaclog3.head()

#adding an elapsed time column
vac_timestamps3=pd.to_datetime(vaclog3["Time"],format="%d/%m/%Y %H:%M:%S")
runtime3 = (vac_timestamps3-vac_timestamps3[0]).dt.total_seconds()    
vaclog3.insert(2,"Elapsed time",runtime3)
# vaclog3.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog3["Time"] = vac_timestamps3.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog3["Time"] = pd.to_datetime(vaclog3["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog3

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2022-11-30 14:56:08,0.0,0.50259,5.290000e-08,5.210000e-08,2.804000e-08,6.246000e-07,0.000606,0.0,-31.986,164.646,NaN,NaN
1,NaN,2022-11-30 14:56:15,7.0,0.50226,5.320000e-08,5.240000e-08,2.828000e-08,6.246000e-07,0.000616,0.0,-32.099,164.758,NaN,NaN
2,NaN,2022-11-30 14:56:22,14.0,0.50199,5.350000e-08,5.280000e-08,2.842000e-08,6.246000e-07,0.000607,0.0,-32.435,164.646,NaN,NaN
3,NaN,2022-11-30 14:56:29,21.0,0.49881,5.390000e-08,5.310000e-08,2.866000e-08,6.238000e-07,0.000612,0.0,-32.435,164.758,NaN,NaN
4,NaN,2022-11-30 14:56:37,29.0,0.50170,5.420000e-08,5.350000e-08,2.886000e-08,6.246000e-07,0.000612,0.0,-32.323,164.758,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,NaN,2022-11-30 18:41:53,13545.0,0.49391,8.130000e-09,8.560000e-09,7.903000e-09,5.573000e-07,0.000610,263.0,-5.163,4287.637,NaN,NaN
1887,NaN,2022-11-30 18:42:00,13552.0,0.49383,8.110000e-09,8.550000e-09,7.893000e-09,5.573000e-07,0.000607,262.0,-5.163,4287.637,NaN,NaN
1888,NaN,2022-11-30 18:42:07,13559.0,0.49425,8.100000e-09,8.590000e-09,7.884000e-09,5.573000e-07,0.000621,264.0,-5.275,4287.749,NaN,NaN
1889,end,2022-11-30 18:42:14,13566.0,0.49413,8.080000e-09,8.590000e-09,7.893000e-09,5.573000e-07,0.000620,263.0,-5.275,4287.637,NaN,NaN


In [5]:
#throw out vaclog3 values from elapsed time 3945-6683s
missing_vals = vaclog3.loc[(vaclog3["Elapsed time"] >= 3945.5) & (vaclog3["Elapsed time"] <= 6683)]
vaclog3.drop(index=missing_vals.index,inplace=True)

## Math

In [6]:
#divide barion 2 readings by dual experiment readings for vaclog 1 and 2
dtb_1 = vaclog1["Barion_2"]/vaclog1["DUAL experiment"]
dtb_2 = vaclog2["Barion_2"]/vaclog2["DUAL experiment"]

#set cutoff point 1.0 for when DUAL gauge goes under range

dtb_1.clip(lower=1, inplace=True)
dtb_2.clip(lower=1, inplace=True)

print(dtb_1.unique())
print(dtb_2)



[1.         1.51229318 1.11603189 ... 2.18466612 2.18985642 2.18223188]
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
3482    1.0
3483    1.0
3484    1.0
3485    1.0
3486    1.0
Length: 3487, dtype: float64


In [7]:
#insert the factors into the respected dataframes 

vaclog1.insert(loc=14, column="dtb_1", value=dtb_1)
vaclog2.insert(loc=14, column="dtb_2", value=dtb_2)


In [8]:
#first, reindex vaclog1 and vaclog2 to vaclog 3 
vaclog1 = vaclog1.reindex(vaclog3.index, method='nearest', limit=1)
vaclog2 = vaclog2.reindex(vaclog3.index, method='nearest', limit=1)

#multiply the calculated factor with Dual gauge reading from vaclog3
barion_estimate1 = (vaclog1["dtb_1"]*vaclog3["DUAL experiment"])
barion_estimate2 = (vaclog2["dtb_2"]*vaclog3["DUAL experiment"])

#checking for NaNs and dropping them

barion_estimate1_clean = barion_estimate1.dropna()
barion_estimate2_clean = barion_estimate2.dropna()

print(vaclog3["Barion_2"].isnull().values.any(), barion_estimate2.isnull().values.any())
print(barion_estimate2_clean.isnull().values.any(), barion_estimate1_clean.isnull().values.any())


True True
False False


In [9]:
#store the dropped rows to remove them also from vaclog 3 later
only_na1 = barion_estimate1[~barion_estimate1.index.isin(barion_estimate1_clean.index)]
only_na2 = barion_estimate2[~barion_estimate2.index.isin(barion_estimate2_clean.index)]

print(only_na1.index,"\n",only_na2)
print(vaclog3["Barion_2"])

#drop the bad rows from vaclog 1-3 as well
vaclog3.drop(index=only_na1.index,inplace=True)
vaclog1.drop(index=only_na1.index,inplace=True)
vaclog2.drop(index=only_na1.index,inplace=True)
vaclog3_clean = vaclog3.drop(index=vaclog3.index[186],axis=0)
print(vaclog3["Barion_2"].isnull().values.any())

#calculate the error compared to measured BA2 
acc1 = r2_score(vaclog3["Barion_2"], barion_estimate1_clean)
acc2 = r2_score(vaclog3_clean["Barion_2"], barion_estimate2_clean)



Int64Index([556], dtype='int64') 
 186   NaN
556   NaN
dtype: float64
0       5.290000e-08
1       5.320000e-08
2       5.350000e-08
3       5.390000e-08
4       5.420000e-08
            ...     
1886    8.130000e-09
1887    8.110000e-09
1888    8.100000e-09
1889    8.080000e-09
1890    8.090000e-09
Name: Barion_2, Length: 1510, dtype: float64
False


In [10]:
print(acc1, acc2)

0.9073153069722542 0.8096910695282026


## Fitting the correction factor vs time 

## Plotting

In [11]:
#Plotting real BA2 vs estimated BA2
plt.figure(figsize=(12,6))
plt.plot(vaclog3["Elapsed time"],vaclog3["Barion_2"],marker=".", markersize=4,label='real Barion 2')
plt.plot(vaclog3_clean["Elapsed time"],barion_estimate2_clean,marker=".", markersize=4,label='vaclog 2 estimated Barion 2')
plt.plot(vaclog3["Elapsed time"],barion_estimate1_clean,marker=".", markersize=4,label='vaclog 1 estimated Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Real BA2 data vs estimated data based on DUAL gauge* correction factor")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')

plt.savefig(r'./graphs/real vs estimated BA2.png')
plt.show()

In [12]:
#correction factor vs pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog3["Barion_2"],vaclog3["Barion_2"],marker=".", markersize=4,label='real Barion 2')
plt.plot(vaclog3["Barion_2"],vaclog1["dtb_1"],marker=".", markersize=4,label='vaclog 1 dual to barion coef')
plt.plot(vaclog3["Barion_2"],vaclog2["dtb_2"],marker=".", markersize=4,label='vaclog 2 dual to barion coef')

plt.xlabel('Pressure (mbar)')
plt.ylabel('Correction factor (mbar/mbar)')
plt.legend()
plt.title("Real BA2 data vs estimated data based on DUAL gauge* correction factor")
ax = plt.gca()
ax.set_yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=True, which='both', axis='both')

plt.savefig(r'./graphs/correction factor vs pressure.png')
plt.show()

In [13]:
#correction factor vs time
plt.figure(figsize=(12,6))
plt.plot(vaclog3["Elapsed time"],vaclog1["dtb_1"],marker=".", markersize=4,label='coef based on vaclog 1 data')
plt.plot(vaclog3["Elapsed time"],vaclog2["dtb_2"],marker=".", markersize=4,label='coef based on vaclog 2 data')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Correction factor (mbar/mbar)')
plt.legend()
plt.title("Real BA2 data vs estimated data based on DUAL gauge* correction factor")
ax = plt.gca()
ax.set_yscale('linear')
plt.grid(visible=True,which='both', axis='both')

plt.savefig(r'./graphs/correction factor vs time.png')
plt.show()